# Open-CV Course - Full torurial with python
this is a Youtube video from FreeCodeCamp , this Project is not working cause of old library
- Youtube Video - https://www.youtube.com/watch?v=oXlwWbU8l2o
- Course Github Repo - https://github.com/jasmcaus/opencv-course
- Open-CV Github Repo - https://github.com/opencv/opencv




In [ ]:
!pip install caer canaro

## Import Dependencies

In [ ]:
import os 
import caer 
import numpy as np
import cv2 as cv
import gc
import canaro

In [ ]:
IMG_SIZE = (80,80)
BATCH_SIZE = 32
EPOCHS = 10
channels = 1 # num of collers
char_path = r'/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset'

## Selecting the Top 10 folder that big num of images

In [ ]:
# Go to every folder and get folder name and num of images in it
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
    
# Sort the data in Descending order
char_dict = caer.sort_dict(char_dict , descending = True)

# Save top 10 Charachters
characters = []
for i in char_dict[:10]:
    characters.append(i[0])

In [ ]:
# Create the training data
train = caer.preprocess_from_dir(char_path , charachters , channels = channels , IMG_SIZE = IMG_SIZE , isShuffle = True)
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0] , cmap = 'gray')
plt.show()

In [ ]:
featureSet , lablers = caer.sep_train(train , IMG_SIZE = IMG_SIZE)

## Preprocessing the Data

### Normalizing the data
to be faster to process

In [ ]:
from tensorflow.keras.utils import to_categorical

# Normalize the feature set ==> (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(lablers , len(characters))

### Spliting the Data

In [ ]:
x_train , x_val , y_train , y_val = caer.train_val_split(featureSet , labels , val_ratio = 0.2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
# Image Data Genarator
datagen = canaro.generators.imageDataGenerator()
train_get = datagen.flow(x_train , y_train , batch_size = BATCH_SIZE)

## Creating the Model
> we have a problem here is that the `canaro` lib is old

In [ ]:
# Creating the Model
model = canaro.models.createSimpsonsModel(IMG_SIZE = IMG_SIZE , channels = channels , output_dim = len(characters) , loss = 'binary_crossentropy ' , learning_rate =0.001 ,momentum = 0.9 , nesterov = True)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(train_gen , steps_per_wpoch = len(x_train)//BATCH_SIZE , epochs = EPOCHS , validation_data = (x_val,y_val) , validation_steps= len(y_val)//BATCH_SIZE , callsbacks = callbacks_list )

## Test the model usign OpenCV


In [ ]:
test_path = r'/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_20.jpg'
img = cv.imread(test_path)
plt.imshow(img , cmap = 'gray')
plt.show()

In [ ]:
def prepare(img):
    img = cv.cvtColor(img , cv.COLOR_BGR2GRAY)
    img = cv.resize(img,IMG_SIZE)
    img.caer.reshape(img,IMG_SIZE , 1)
    reuturn img

In [ ]:
predictions = model.predict(prepare(img))
print(characters[np.argmax(predictions[0])])
img = cv.imread(test_path)
plt.imshow(img , cmap = 'gray')
plt.show()